<h1><p style="text-align: center;">Quickstart for MacTrack2</p></h1>

Through this file, you can find all the information you need to make this project your own. Don't forget to create your Python environment with the packages available in the `environment.txt` file.

> WARNING! When you need to make actions, the details will be preceded by this style.

### Table of contents

* [How to create your own model](#chapter1)  
  * [Dataset](#dataset)
  * [Training](#training)
  * [Testing](#testing)
  * [Model summary](#summary)

## How to create your own model <a class="anchor" id="chapter1"></a>

### Dataset <a class="anchor" id="dataset"></a>

First, you need a dataset to create your model and a *models* folder to store it. 

In [ ]:
import os

dirs = [
    "model",
    "model/dataset",
    "model/dataset/train",
    "model/dataset/train/train_x",
    "model/dataset/train/train_y",
    "model/dataset/test",
    "model/dataset/test/test_x",
    "model/dataset/test/test_y",
    "model/models",
]

for d in dirs:
    os.makedirs(d, exist_ok=True)

> This dataset is essential to create your model, you need to add the following :  
> - `train_x` : a training set of images that you hand-cut yourself using [Fiji](https://imagej.net/software/fiji/downloads)
> - `train_y` : the masks for each segmented frame in a zip file containing all the ROI files
> - `test_x` and `test_y` : you should do the same, it will serve to test the model that has been trained on the training set.

Here is an example of an image that will be placed in the *x* sets (wether it is the training or the test set) on the left. And on the right, each yellow lining represent a ROI (region of interest). Those are saved in a zip file and in the *y* sets.  
<p align="center">
  <img src="images/example_frame.jpg" alt="frame_ex" width="45%"/>
  <img src="images/example_frame_ROI.jpg" alt="ROI_ex" width="45%"/>
</p>


(TODO : DOC ABOUT FIJI)

Next, the `kartezio` package that will help us build the model need two other files : `META.json` and `dataset.csv`.

The META file is here to help the package recognize the formats of the different objects in the dataset folder.  
> You need to run the following to help you create it.

In [ ]:
import json

# Path to the JSON file
meta_path = "model/dataset/META.json"

# Content of the JSON file
meta_data = {
    "name": "Macrophage",
    "scale": 1.0,
    "label_name": "macrophage",
    "mode": "dataframe",
    "input": {
        "type": "image",
        "format": "hsv"
    },
    "label": {
        "type": "roi",
        "format": "polygon"
    }
}

# Create the JSON file with the content in it
with open(meta_path, "w", encoding="utf-8") as f:
    json.dump(meta_data, f, indent=4)

print(f"Fichier {meta_path} créé avec succès !")

The csv file is here to help kartezio read your dataset.

In [ ]:
from Set_up.dataset_csv import create_dataset_csv
create_dataset_csv(input_folder='model/dataset', output_csv='model/dataset/dataset.csv')

Your two files are now created and located in the `model/dataset` folder. We can now train our model.

### Training <a class="anchor" id="training"></a>

Now that we have the structure kartezio needs to function correctly, we can create and train a model using the `create_segmentation_model` function of the kartezio package. You can find the following code in the `Set_up/train_model.py` file.

In [ ]:
from kartezio.apps.segmentation import create_segmentation_model
from kartezio.endpoint import EndpointThreshold
from kartezio.dataset import read_dataset
from kartezio.training import train_model

DATASET = "model/dataset"
OUTPUT = "model/models"

generations = 1000
_lambda = 5
frequency = 5
rate = 0.1
print(rate)
model = create_segmentation_model(
    generations,
    _lambda,
    inputs=3,
    nodes=30,
    node_mutation_rate=rate,
    output_mutation_rate=rate,
    outputs=1,
    fitness="IOU",
    endpoint=EndpointThreshold(threshold=4)
)

dataset = read_dataset(DATASET)
elite, a = train_model(model, dataset, OUTPUT, callback_frequency=frequency)

### Testing <a class="anchor" id="testing"></a>

Now that you trained your model, you can test it to see if it gives good prediction.

In [ ]:
import numpy as np
import pandas as pd

from kartezio.easy import print_stats
from kartezio.dataset import read_dataset
from kartezio.fitness import FitnessIOU
from kartezio.inference import ModelPool

scores_all = {}
pool = ModelPool(f"model/models", FitnessIOU(), regex="*/elite.json").to_ensemble()
dataset = read_dataset(f"model/dataset", counting=True)
annotations_test = 0
annotations_training = 0
roi_pixel_areas = []
for y_true in dataset.train_y:
    n_annotations = y_true[1]
    annotations_training += n_annotations
for y_true in dataset.test_y:
    annotations = y_true[0]
    n_annotations = y_true[1]
    annotations_test += n_annotations
    for i in range(1, n_annotations + 1):
        roi_pixel_areas.append(np.count_nonzero(annotations[annotations == i]))
print(f"Total annotations for training set: {annotations_training}")
print(f"Total annotations for test set: {annotations_test}")
print(f"Mean pixel area for test set: {np.mean(roi_pixel_areas)}")


scores_test = []
scores_training = []
for i, model in enumerate(pool.models):
    # Test set
    _, fitness, _ = model.eval(dataset, subset="test")
    scores_test.append(1.0 - fitness)

    # Training set
    _, fitness, _ = model.eval(dataset, subset="train")
    scores_training.append(1.0 - fitness)


scores_all[f"training"] = scores_training
scores_all[f"test"] = scores_test
print_stats(scores_training, "IOU", "training set")
print_stats(scores_test, "IOU", "test set")

pd.DataFrame(scores_all).to_csv("model/scores.csv", index=False)

### Model summary <a class="anchor" id="summary"></a>

You can summarize your trained model to understand its structure and behavior. The summary provides insights into the nodes, connections, and parameters of the model. This can help you analyze how the model was built and how it processes data.


In [ ]:
from Set_up.explain_model import summary_model

summary = summary_model('model')

You can create a `csv` file for the nodes that will help you understand better the model build.

In [ ]:
summary.to_csv()

You can also access other parameters that are not displayed in the summary.

In [ ]:
print(summary.keys())

And you can extract any object from that list.

In [ ]:
print(summary.endpoint)